# LCA-PrOMMiS Integration
This notebook examines how to interface openLCA with [PrOMMiS](https://prommis.readthedocs.io/en/latest/) for critical mineral [leaching](https://github.com/prommis/prommis/tree/main/src/prommis/leaching) of coal refuse.


In [ ]:
# Import methods from the leaching flowsheet to build model.
from prommis.leaching.leach_flowsheet import build_model
from prommis.leaching.leach_flowsheet import set_inputs
from prommis.leaching.leach_flowsheet import set_scaling

# Import

In [ ]:
# Build the model.
m = build_model()
set_inputs(m)
set_scaling(m)

In [ ]:
# Create a scaled version of the model to solve
scaling = TransformationFactory("core.scale_model")
scaled_model = scaling.create_using(m, rename=False)

# Initialize model
initializer = LeachingTrainInitializer()
initializer.initialize(scaled_model.fs.leach)

# Solve scaled model
solver = SolverFactory("ipopt")
solver.solve(scaled_model, tee=True)

# Propagate results back to unscaled model
scaling.propagate_solution(scaled_model, m)

# Display some results
m.fs.leach.liquid_outlet.display()
m.fs.leach.solid_outlet.display()

m.fs.leach.report()

In [ ]:
dir(m)

In [ ]:
m.pprint()

# Appendix

## Learning Pyomo
To understand the underlying optimization for PrOMMiS.
See [here](https://jckantor.github.io/ND-Pyomo-Cookbook/notebooks/01.02-Running-Pyomo-on-Google-Colab.html).

In [ ]:
from pyomo.environ import ConcreteModel
from pyomo.environ import NonNegativeReals
from pyomo.environ import Constraint
from pyomo.environ import Var
from pyomo.environ import Objective
from pyomo.environ import maximize
from pyomo.environ import SolverFactory

# create a model
model = ConcreteModel()

# declare decision variables
model.x = Var(domain=NonNegativeReals)
model.y = Var(domain=NonNegativeReals)

# declare objective
model.profit = Objective(expr = 40*model.x + 30*model.y, sense=maximize)

# declare constraints
model.demand = Constraint(expr = model.x <= 40)
model.laborA = Constraint(expr = model.x + model.y <= 80)
model.laborB = Constraint(expr = 2*model.x + model.y <= 100)

model.pprint()

def display_solution(model):

    # display solution
    print('\nProfit = ', model.profit())

    print('\nDecision Variables')
    print('x = ', model.x.value)
    print('y = ', model.y())

    print('\nConstraints')
    print('Demand  = ', model.demand())
    print('Labor A = ', model.laborA())
    print('Labor B = ', model.laborB())

In [ ]:
SolverFactory('clp').solve(model, tee=True).write()

display_solution(model)

In [ ]:
SolverFactory('ipopt').solve(model, tee=True).write()

display_solution(model)